In [1]:
import s4sim.hardware.config
c=s4sim.hardware.config.get_example()

In [2]:
import s4sim

In [3]:
s4sim.__version__

'0+untagged.115.gc57d15d'

In [4]:
c.data["telescopes"]["SAT0"]["fwhm"]

OrderedDict([('LFS1', 72.8),
             ('LFS2', 72.8),
             ('MFLS1', 25.5),
             ('MFLS2', 25.5),
             ('MFHS1', 22.7),
             ('MFHS2', 22.7),
             ('HFS1', 13.0),
             ('HFS2', 13.0)])

In [5]:
c.data["telescopes"]["SAT3"]["fwhm"].keys()

odict_keys(['LFS1', 'LFS2', 'MFLS1', 'MFLS2', 'MFHS1', 'MFHS2', 'HFS1', 'HFS2'])

In [6]:
c.data["telescopes"]["SAT3"].keys()

odict_keys(['tubes', 'platescale', 'tubespace', 'fwhm'])

In [7]:
c.data["bands"].keys()

odict_keys(['ULFL1', 'LFL1', 'LFL2', 'LFS1', 'LFS2', 'MFL1', 'MFL2', 'MFLS1', 'MFLS2', 'MFHS1', 'MFHS2', 'HFL1', 'HFL2', 'HFS1', 'HFS2'])

In [8]:
c.data["bands"]["LFL1"]

OrderedDict([('center', 27.0),
             ('low', 24.0),
             ('high', 30.0),
             ('bandpass', ''),
             ('NET', 387.0),
             ('fknee', 50.0),
             ('fmin', 0.01),
             ('alpha', 3.5),
             ('A', 0.09),
             ('C', 0.87)])

In [9]:
import h5py

In [10]:
h5py.get_config().track_order = True

In [11]:
s4 = h5py.File("../cmbs4_tophat.h5", "w")

In [12]:
import numpy as np

# NET and number of detectors per tube

From the spreadsheet at https://docs.google.com/spreadsheets/d/1X0x8wYhjHdI_WPjhgbtyVlVQrX86tTgxgqEUhan2Hxg/edit?pli=1#gid=1404929524

Extracted on July 2nd 2020

In [13]:
NUM_DET = {}
SAT_NUM_DET_STRING = "296	296	3556	3556	3556	3556	8594	8594"
for sat_band, num_det in zip(c.data["telescopes"]["SAT3"]["fwhm"].keys(), SAT_NUM_DET_STRING.split()):
    NUM_DET[sat_band] = int(num_det)

In [14]:
NET = {}
SAT_NET_STRING = "177	224	270	309	238	331	747	1281"
for band, net in zip(c.data["telescopes"]["SAT3"]["fwhm"].keys(), SAT_NET_STRING.split()):
    NET[band] = float(net)

In [15]:
LAT_NUM_DET_STRING = "216	384	384	3456	3456	3456	3456"
for sat_band, num_det in zip(c.data["telescopes"]["LAT0"]["fwhm"].keys(), LAT_NUM_DET_STRING.split()):
    NUM_DET[sat_band] = int(num_det)

In [16]:
c.data["telescopes"]["LAT0"]["fwhm"].keys()

odict_keys(['ULFL1', 'LFL1', 'LFL2', 'MFL1', 'MFL2', 'HFL1', 'HFL2'])

In [17]:
LAT_NET_STRING = "473.0	429.0	261.0	305.0	385.0	792.0	1990.0"
for band, net in zip(c.data["telescopes"]["LAT0"]["fwhm"].keys(), LAT_NET_STRING.split()):
    NET[band] = float(net)

In [18]:
for i, (band, v) in enumerate(c.data["bands"].items()):
    print("## " + band)
    print("Bandpass [GHz] low,center,high: ", v["low"], v["center"], v["high"])
    try:
        tel = "SAT"
        fwhm = c.data["telescopes"][tel + "0"]["fwhm"][band]
    except:
        tel = "LAT"
        fwhm = c.data["telescopes"][tel + "0"]["fwhm"][band]
    print("Beam [arcmin]: ", fwhm)
    s4.create_group(band)
    s4[band].attrs["band"] = band
    s4[band].attrs["telescope"] = tel
    s4[band].attrs["center_frequency_GHz"] = v["center"]
    s4[band].attrs["fwhm_arcmin"] = fwhm
    s4[band].attrs["detectors_per_tube"] = NUM_DET[band]
    s4[band].attrs["NET_uKsqrts"] = NET[band]
    s4[band].attrs["index"] = i
    s4[band].create_dataset('bandpass_frequency_GHz', data=np.linspace(v["low"], v["high"], 10))
    s4[band].create_dataset("bandpass_weight", data=np.ones(10, dtype=np.float))
    print("Num det per tube", s4[band].attrs["detectors_per_tube"])

## ULFL1
Bandpass [GHz] low,center,high:  17.5 20.0 22.5
Beam [arcmin]:  10.0
Num det per tube 216
## LFL1
Bandpass [GHz] low,center,high:  24.0 27.0 30.0
Beam [arcmin]:  7.4
Num det per tube 384
## LFL2
Bandpass [GHz] low,center,high:  30.0 39.0 48.0
Beam [arcmin]:  5.1
Num det per tube 384
## LFS1
Bandpass [GHz] low,center,high:  25.5 30.0 34.5
Beam [arcmin]:  72.8
Num det per tube 296
## LFS2
Bandpass [GHz] low,center,high:  34.0 40.0 46.0
Beam [arcmin]:  72.8
Num det per tube 296
## MFL1
Bandpass [GHz] low,center,high:  75.5 93.0 110.5
Beam [arcmin]:  2.2
Num det per tube 3456
## MFL2
Bandpass [GHz] low,center,high:  125.0 145.0 165.0
Beam [arcmin]:  1.4
Num det per tube 3456
## MFLS1
Bandpass [GHz] low,center,high:  74.8 85.0 95.2
Beam [arcmin]:  25.5
Num det per tube 3556
## MFLS2
Bandpass [GHz] low,center,high:  129.1 145.1 161.0
Beam [arcmin]:  25.5
Num det per tube 3556
## MFHS1
Bandpass [GHz] low,center,high:  83.6 95.0 106.4
Beam [arcmin]:  22.7
Num det per tube 3556
## MFHS

In [19]:
s4.keys()

<KeysViewHDF5 ['ULFL1', 'LFL1', 'LFL2', 'LFS1', 'LFS2', 'MFL1', 'MFL2', 'MFLS1', 'MFLS2', 'MFHS1', 'MFHS2', 'HFL1', 'HFL2', 'HFS1', 'HFS2']>

In [20]:
s4["MFL1"].get("bandpass_frequency_GHz")

<HDF5 dataset "bandpass_frequency_GHz": shape (10,), type "<f8">

In [21]:
s4.close()

In [22]:
pwd

'/global/project/projectdirs/planck/software/zonca/software/s4mapbasedsims/202006_foregrounds_extragalactic_cmb_tophat/utils'